# Lista de Exercícios 4: Gradiente de Política

#### Disciplina: Aprendizado por Reforço
#### Professor: Luiz Chaimowicz
#### Monitores: Marcelo Lemos e Ronaldo Vieira

---

## Instruções

- Leia atentamente toda a lista de exercícios e familiarize-se com o código fornecido **antes** de começar a implementação.
- Os locais onde você deverá escrever suas soluções estão demarcados com comentários `# YOUR CODE HERE` ou `YOUR ANSWER HERE`.
- **Não altere o código fora das áreas indicadas, nem adicione ou remova células. O nome deste arquivo também não deve ser modificado.**
- Antes de submeter, certifique-se de que o código esteja funcionando do início ao fim sem erros.
- Submeta apenas este notebook (*ps4.ipynb*) com as suas soluções no Moodle.
- Prazo de entrega: 01/07/2025. **Não serão aceitas submissões fora do prazo.**
- ***SUBMISSÕES QUE NÃO SEGUIREM ESTAS INSTRUÇÕES NÃO SERÃO AVALIADAS.***
- Utilize a [documentação do Gymnasium](https://gymnasium.farama.org/) para auxiliar sua implementação.
- Em caso de dúvidas entre em contato pelo fórum "Dúvidas com relação aos exercícios e trabalho de curso" no moodle da Disciplina.

---

## Cart Pole

Cart Pole é um ambiente no qual o agente precisa equilibrar um pêndulo invertido acoplado a um carrinho que se move horizontalmente. O objetivo é evitar que o pêndulo caia, controlando a posição do carrinho de maneira apropriada. O desafio está em manter o equilíbrio do sistema aplicando forças para a esquerda ou para a direita, de modo a contrabalançar a inclinação do pêndulo.

![Cart Pole](https://gymnasium.farama.org/_images/cart_pole.gif)

O espaço de observação é contínuo e composto por quatro variáveis: a posição e a velocidade do carrinho, o ângulo do pêndulo e a velocidade angular do pêndulo. O agente pode escolher entre duas ações discretas: aplicar uma força no carrinho para a esquerda ou para a direita. O estado inicial de cada episódio é definido de forma aleatória, com todos os valores dentro do intervalo $(−0.05, 0.05)$. O sempre sistema começa em uma configuração próxima do equilíbrio vertical, mas com pequenas variações.

O episódio termina quando o ângulo do pêndulo em relação ao eixo vertical excede 12 graus, quando o carrinho se afasta mais de 2.4 unidades de distância do centro, ou quando o limite de 500 passos é atingido. A cada passo, o agente recebe uma recompensa de $1$, incentivando-o a manter o sistema estável pelo maior tempo possível. Para mais detalhes sobre o ambiente leia a [documentação do gymnasium](https://gymnasium.farama.org/environments/classic_control/cart_pole/).

Nesta lista de exercícios, você irá trabalhar com o ambiente Cart Pole descrito acima. Seu objetivo será implementar o algoritmo REINFORCE para solucionar este problema.

## REINFORCE

Nesta atividade, você implementará um agente baseado no algoritmo REINFORCE, utilizando uma **política linear com softmax**. Não é permitido o uso de bibliotecas de aprendizado profundo. Você será responsável por definir e inicializar os pesos da política, calcular o gradiente da função objetivo e realizar a atualização dos parâmetros ao longo do treinamento.

Siga as instruções abaixo para implementar seu agente:

1. Implemente o método `__init__` que inicializa um novo agente. Ele deve receber como parâmetros o espaço de observações, o espaço de ações, a taxa de aprendizado $\alpha$, e o fator de desconto $\gamma$.
2. Implemente o método `softmax`, que recebe um vetor de preferências (logits) $z = (z_1,\dotsc,z_K) \in \mathbb{R}^K$ e retorna as probabilidades de cada ação segundo a função softmax $\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$, para $i = 1, \dotsc, K.$
3. Implemente o método `choose_action`, responsável por escolher uma ação a partir de um estado observado.
4. Implemente o método `learn`, que atualiza os pesos do agente com base na experiência obtida durante a interação com o ambiente.
5. Implemente o método `train`, que executa o loop de treinamento do algoritmo REINFORCE. O ambiente de treinamento e o número de episódios devem ser fornecidos como parâmetros de entrada. O método deve retornar uma lista com a soma das recompensas obtidas ao longo de cada episódio.

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class ReinforceAgent:
    def __init__(self, observation_space, action_space, alpha, gamma):
        # YOUR CODE HERE
        raise NotImplementedError()

    def softmax(self, z):
        # YOUR CODE HERE
        raise NotImplementedError()

    def choose_action(self, state):
        # YOUR CODE HERE
        raise NotImplementedError()

    def learn(self, trajectories):
        # YOUR CODE HERE
        raise NotImplementedError()

    def train(self, env, episodes):
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# Não altere ou remova esta célula

6. Treine um novo agente no ambiente Cart Pole por 2000 episódios. Escolha parâmetros $\alpha$ e $\gamma$ de acordo com o desempenho do seu modelo. Armazene os retornos episódicos, obtidos no método train, na variável `reinforce_returns`.

In [ ]:
env = gym.make("CartPole-v1")

# YOUR CODE HERE
raise NotImplementedError()

env.close()

In [ ]:
# Não altere ou remova esta célula

## Análise

7. Utilize a biblioteca matplotlib para gerar um gráfico dos retornos episódicos obtidos pelo agente, armazenados na variável `reinforce_returns` no exercício anterior. Caso os dados apresentem muita variação, aplique uma média móvel para suavizar a curva e evidenciar melhor a tendência de aprendizado. Adicione um título e rótulos de eixos apropriados para facilitar a interpretação.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

8. Durante o treinamento, o seu agente apresentou instabilidades, como variações bruscas na recompensa ou dificuldade em convergir? Que estratégias poderiam ser adotadas para mitigar instabilidades neste algoritmo?

YOUR ANSWER HERE

9. Explique qual é o papel da função softmax na definição da política. Por que, nesse contexto, não é suficiente escolher apenas a ação com maior valor?

YOUR ANSWER HERE

10. Por que o algoritmo REINFORCE precisa aguardar a conclusão de todo o episódio antes de realizar atualizações nos parâmetros da política?

YOUR ANSWER HERE